# ARM Computer Exercise 1

In this exercise, we compare the RiskMetrics VaR to the (Weighted) Historical Simulation approach. We compare the time patterns of the three VaR measures applied to S&P 500 index returns in the period January 2001 through December 2010. Next, we evaluate the three approaches using backtests.

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os

Efficient programming of the Weighted Historical Simulation method requires a function that can deliver weighted percentiles from a sample. The code for this has been found on the web, and imported here.

In [2]:
def weighted_quantile(values, quantiles, sample_weight=None, values_sorted=False, old_style=False):
    """ Source: https://stackoverflow.com/questions/21844024/weighted-percentile-using-numpy
    Very close to np.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: np.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of initial array
    :param old_style: if True, will correct output to be consistent with np.percentile.
    :return: np.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), 'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

Data loading, using pandas datareader connected to Yahoo! Finance.
Note that after creating the returns series R:
* observations 0:250 refer to year 2000, used to start up (W)HS
* observations 251:2765 refer to 2001-2010, evaluation sample

In [3]:
st = dt.datetime(2000, 1, 1)
en = dt.datetime(2010, 12, 31)
data = web.DataReader('^GSPC', 'yahoo', start=st, end=en)
S = data['Adj Close']
R = 100 * np.log(1 + S.pct_change().dropna())
R.name = 'R'

In [4]:
R

Date
2000-01-04   -3.909918
2000-01-05    0.192034
2000-01-06    0.095522
2000-01-07    2.672995
2000-01-10    1.112782
                ...   
2010-12-27    0.061251
2010-12-28    0.077103
2010-12-29    0.100864
2010-12-30   -0.150936
2010-12-31   -0.019081
Name: R, Length: 2766, dtype: float64

When the above code does not run properly, replace the previous block by the following, which loads the data from the provided csv file; otherwise, skip this block.

In [5]:
path = 'C:\Users\Jinhyun\Documents\GitHub\Advanced Risk Management\Tutorial\Week 1'    # change path to your working directory
os.chdir(path)
dframe = pd.read_csv('SP500.csv', parse_dates=True, index_col='Date')
S = dframe['SP500']
R = 100 * np.log(1 + S.pct_change().dropna())
R.name = 'R'

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_4748/1310708235.py, line 1)

### Question 1

Calculate the 1%, one-day VaR for S&P 500 index returns for each day in the evaluation period (January 2001 through December 2010), using each of the three methods, and keep the result in three vectors / arrays. To get started, this code below already contains the definition of a weight vector w, based on a historical period m = 250, and a parameter $\lambda = \eta = 0.94$ for the WHS and RM methods. Note that WHS and the $\sigma_{t+1}$ sequence in RM need a start-up sample period, for which we use data from the year 2000 (252 observations).

*Note*: an easy way to implement the RM method to construct $\sigma_t^2$ is to use the `series.ewm()` function from `pandas`. Note that in that function, $\alpha = 1 - \lambda$, and also that you need to apply this to the *lagged* squared return (so using the `series.shift()` function).

In [ ]:
p = 0.01
labda = 0.94
m = 250
eta = labda
tau = np.arange(m,0,-1)
w = eta**tau
w /= sum(w)          # Weights are sorted from low to high

### Question 2

Make a figure where each of the three VaR measures are plotted against time. Discuss the similarities and differences.

### Question 3

Investigate the effect of changing the eta ($\eta$) parameter in the WHS method: what happens if
we give eta a value very close to 1, e.g. 0.999?

### Question 4

Construct the hit sequences $I_{t+1} = \mathbb{I}( R_{t+1} < -VaR_{t+1}^{0.01})$, for each of the three VaR methods.
Next, test unconditional coverage and independence using the methods described in the slides (you may also use Christoffersen’s LR tests for comparison). What do you conclude?

(*Note*: $\mathbb{I}$ is the indicator function, so $\mathbb{I}(A) = 1$ if $A$ is true, and $0$ otherwise.)